# Deutch-Jozsa Algorithm
DJ is a quantum algorithm that exhibits an exponential speedup over classical implementation. Given n-qubits for n-number of parameters, its time complexity is O(1) compared to the worst case on classical implementation which is (O)2^(n-1). Though it has no practical application, it clearly illustrates the quantum mechanical effects of superposition, interference and entanglement. 

In [ ]:
// NOTE: Namespace not accepted due to the non-contiguous nature of cells
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert as Convert;
open Microsoft.Quantum.Measurement; 
open Microsoft.Quantum.Arrays;

## Oracles
Oracles are the functions which property we want to determine. Note the use of [ancilla](https://en.wikipedia.org/wiki/Ancilla_bit) qubit to respresent the result. This is a requirement in [reversible computing](https://en.wikipedia.org/wiki/Reversible_computing) like QC.

In [ ]:
operation Const0(qubits: Qubit[]) : Unit {
    ApplyToEach(I, qubits); // You may omit
}

operation Const1(qubits: Qubit[]) : Unit {       
    I(qubits[0]);  // You may omit
    X(qubits[1]);      
}

operation Balanced0(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[1]);
}

operation Balanced1(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[1]);
    X(qubits[1]);
}

## Circuit
To accomodate different oracles, we isolate the circuit creation in its own operation. Note that Q# has no support on named delegate, only lambda.

In [ ]:
operation Deutch(oracle: ((Qubit[]) => Unit)) : Result {
    use qubits = Qubit[2] {

        X(qubits[1]);

        H(qubits[0]);
        H(qubits[1]);

        oracle(qubits);

        H(qubits[0]);

        let res = M(qubits[0]);

        ResetAll(qubits);

        return res;
    }   
}

In [ ]:
@EntryPoint()
operation DJMain() : Result[] {

    mutable res  = [Zero,Zero,Zero,Zero];
    let oracles = [Const0, Const1, Balanced0, Balanced1];

    for i in 0 .. Length(oracles)-1 {            
        set res w/= i <- Deutch(oracles[i]);          
    }               

    return res;
}

## Magic commands
[Magic commands](https://docs.microsoft.com/en-us/qsharp/api/iqsharp-magic/) have the advantage of being succint unlike their CLI counterparts.

### Simulation
Prior to submitting the cicuit to a real QC, you may want to simulate it first. The syntax couldn't be simpler

In [ ]:
%simulate DJMain

### Submitting to QPU
Submission to a QPU (Quantum Processing Unit) or real quantum computer involves the followings steps:
1. Connect to your workspace
2. Select target (simulator can be targeted too)
3. Submit the job (non-blocking) / Execute the job (blocking)
4. If you chose the non-blocking way, you can check the status of the job

In [ ]:
%azure.connect "<resource id here which you can find in the Overview tab of your Quantum Workspace"

In [ ]:
%azure.target ionq.qpu

In [ ]:
%azure.submit DJMain jobName="<arbitrary job name>" shots="1024"

In [ ]:
%azure.job "<job id here>"